In [10]:
import tensorflow_datasets as tfds
import tensorflow as tf
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from spark_tensorflow_distributor import MirroredStrategyRunner

In [2]:
# Taken from https://www.tensorflow.org/tutorials/distribute/multi_worker_with_keras

def train():
    BUFFER_SIZE = 10000
    BATCH_SIZE = 64

    def make_datasets_unbatched():
        # Scaling MNIST data from (0, 255] to (0., 1.]
        def scale(image, label):
            image = tf.cast(image, tf.float32)
            image /= 255
            return image, label
        datasets, info = tfds.load(
            name='mnist',
            with_info=True,
            as_supervised=True,
        )
        return datasets['train'].map(scale).cache().shuffle(BUFFER_SIZE)

    def build_model():
        model = tf.keras.Sequential([
            tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
            tf.keras.layers.MaxPooling2D(),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(10, activation='softmax'),
        ])
        model.compile(
            loss=tf.keras.losses.sparse_categorical_crossentropy,
            optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
            metrics=['accuracy'],
        )
        return model
    
    batch_size = 64 * 8
    
    train_datasets = make_datasets_unbatched().batch(batch_size).repeat()
    options = tf.data.Options()
    options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.DATA
    train_datasets = train_datasets.with_options(options)
    model = build_model()
    model.fit(x=train_datasets, epochs=3, steps_per_epoch=5)
    return tf.config.experimental.list_physical_devices('CPU')

In [12]:
MirroredStrategyRunner(num_slots=2).run(train)

Doing GPU training...


Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.IllegalArgumentException: The number of slots on an executor has to be limited by the number of cores, otherwise you waste resources and dynamic allocation doesn't work properly. Your configuration has core/task cpu slots = 12 and cpu = 1. Please adjust your configuration so that all resources require same number of executor slots.
	at org.apache.spark.SparkContext$.$anonfun$createTaskScheduler$3(SparkContext.scala:2812)
	at org.apache.spark.SparkContext$.$anonfun$createTaskScheduler$3$adapted(SparkContext.scala:2782)
	at scala.collection.mutable.ArraySeq.foreach(ArraySeq.scala:75)
	at org.apache.spark.SparkContext$.checkResourcesPerTask$1(SparkContext.scala:2782)
	at org.apache.spark.SparkContext$.org$apache$spark$SparkContext$$createTaskScheduler(SparkContext.scala:2866)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:528)
	at org.apache.spark.api.java.JavaSp